# NbConvert

## Command line usage

`NbConvert` is the library, and the command line tool that allow to convert from notebook to other formats.
It is a technological preview in 1.0 but is already usable and highly configurable.
It ships already with many default available formats : `html`, `latex`, `markdown`, `python`, `rst` and  `slides`
which are fully base on Jinja templating engine, so writing a converter for your custom format or tweeking the existing 
one should be extra simple.

You can invoke nbconvert by doing

```bash
$ ipython nbconvert <options and arguments>
```

Call `ipython nbconvert` with the `--help` flag or no aruments to get basic help on how to use it.
For more information about configuration use the `--help-all` flag

### Basic export

We will be converting `Converting Notebooks With nbconvert.ipynb'`. 
Be sure to have runed some of the cells in it to have output otherwise you will only see input in nbconvert.
Nbconvert **do not execute the code** in the notebook files, it only converts what is inside.

In [ ]:
%%bash
jupyter nbconvert 'Converting Notebooks With nbconvert.ipynb'

Html is the default value (that can be configured) , so the verbose form would be 

In [ ]:
%%bash
jupyter nbconvert  --to=html 'Converting Notebooks With nbconvert.ipynb'

You can also convert to latex, which will take care of extractin the embeded base64 encoded png, or the svg and call inkscape to convert those svg to pdf if necessary :

In [ ]:
%%bash
jupyter nbconvert  --to=latex 'Converting Notebooks With nbconvert.ipynb'

You should just have to compile the generated `.tex` file. If you get the required packages installed, if should compile out of the box.


In [ ]:
%%bash
jupyter nbconvert  --to=pdf 'Index.ipynb'

Have a look at `04 - Custom Display Logic.pdf`, toward the end, where we compared `display()` vs `display_html()` and returning the object.
See how the cell where we use `display_html` was not able to display the circle, whereas the two other ones were able to select one of the oher representation they know how to display.

### Customizing template

let's look at the first 20 lines of the `python` exporter

In [ ]:
pyfile = !jupyter nbconvert --to python 'Converting Notebooks With nbconvert.ipynb' --stdout
for l in pyfile[20:40]:
    print(l)

We see that the non-code cell are exported to the file. To have a cleaner script, we will export only the code contained in the code cells.

To do so, we will inherit the python template, and overwrite the markdown blocks to be empty.

In [ ]:
%%writefile simplepython.tpl
{% extends 'python.tpl'%}

{% block markdowncell -%}
{% endblock markdowncell %}

## we also want to get rig of header cell
{% block headingcell -%}
{% endblock headingcell %}

## and let's change the appearance of input prompt
{% block in_prompt %}
# This was input cell with prompt number : {{ cell.prompt_number if cell.prompt_number else ' ' }}
{%- endblock in_prompt %}

In [ ]:
pyfile = !jupyter nbconvert --to python 'Converting Notebooks With nbconvert.ipynb' --stdout --template=simplepython.tpl

for l in pyfile[4:40]:
    print(l)
print('...')

I'll let you read Jinja manual for the exact syntax of the template.

## Template that use cells metadata

Notebook fileformat support attaching arbitrary JSON metadata to each cell of a notebook. In this part we will use those metadata.

First you need to choose another notebook you want to convert to html, and tag some of the cell with metadata.
You can see the file `soln/celldiff.js` for a solution on how to tag, or follow the javascript tutorial to see how to do that. Use what we have written there to tag cells of some notebooks to `Easy`|`Medium`|`Hard`|`<None>`, and convert this notebook using your template.

you might need the following : 
```
{% extends 'html_full.tpl'%}
{% block any_cell %}
{{ super() }}
<div style="background-color:red">
<div style='background-color:orange'>
```

`metadata` might not exist, be sure to :

`cell['metadata'].get('example',{}).get('difficulty','')`

tip: use `%%writefile` to edit the template in the notebook :-)

In [ ]:
%%bash
# ipython nbconvert --to=html <your chosen notebook.ipynb> --template=<your template file>

In [ ]:
#%%bash
#jupyter nbconvert --to=html 'Converting Notebooks With nbconvert.ipynb' --template=../../exercises/soln/coloreddiff.tpl

### Get rid of all command line flags.

As of all of IPython nbconvert can be configured using profiles and passing the `--profile` flag. 
Moreover if a `config.py` file exist in current working directory nbconvert will use that, or read the config file you give to it with the `--config=<file>` flag. 

In the end, if you are often running nbconvert on the sam project, `$ ipython nbconvert` should be enough to get you up and ready.

## Cleanup

In [ ]:
!rm -f 'Converting Notebooks With nbconvert.html'
!rm -f 'Converting Notebooks With nbconvert.tex'
!rm -f 'Converting Notebooks With nbconvert.pdf'
!rm -f 'Index.pdf'
!rm -f simplepython.tpl